In [1]:
import pandas as pd
import requests
from bioservices import KEGG
import mygene
import pdb
from chembl_webresource_client.new_client import new_client
from mygene import MyGeneInfo
from tqdm import tqdm
from functools import lru_cache
import pandas as pd

/Users/mjc014/.pyenv/versions/3.12.2/lib/python3.12/site-packages/chembl_webresource_client/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __version__ = __import__('pkg_resources').get_distribution('chembl_webresource_client').version


## Biological Plausibility
In order to provide an additional metric to help review or provide confidence in AI Assisted Literature curation, I am exploring the notion of a **biological plausibility** score. The idea is for any AI extracted drug, gene interaction, assess whether there is any prior knowledge of the drug and gene interaction OR of the drug interacting with any gene pathway containing that gene.

### Load dgiLIT data

In [2]:
df = pd.read_excel('data/final_results_test3.xlsx').drop(labels='Unnamed: 0', axis=1)
print(len(df))
df.head()

137


,pmid,drug_name,gene_name,interaction_occurs_with_gene,interaction_type,evidence,gene_concept,gene_label,gene_match_type,drug_concept,drug_label,drug_match_type
0,37726279,venetoclax,ABCC1,YES,INHIBITING,Genetic and pharmacologic ABCC1 inactivation p...,normalize.gene.hgnc:51,ABCC1,100,normalize.therapy.rxcui:1747556,venetoclax,80
1,37726279,glutathione,ABCC1,YES,ACTIVATING,Consistent with ABCC1-specific export of gluta...,normalize.gene.hgnc:51,ABCC1,100,normalize.therapy.rxcui:4890,glutathione,80
2,37004989,Kynurenine,AhR,YES,ACTIVATING,"An endogenous AhR ligand, kynurenine (Kyn), wa...",normalize.gene.hgnc:348,AHR,100,normalize.therapy.drugbank:DB02070,Kynurenine,80
3,33932119,ONC201,AKT,YES,INHIBITING,"The compensatory, pro-survival PI3K/AKT/mTOR p...",normalize.gene.hgnc:391,AKT1,60,normalize.therapy.iuphar.ligand:9978,ONC201,80
4,26884600,ONC201,AKT,YES,INHIBITING,ONC201 (also called TIC10) is a small molecule...,normalize.gene.hgnc:391,AKT1,60,normalize.therapy.iuphar.ligand:9978,ONC201,80


In [3]:
df[df['drug_name']=='imatinib']

,pmid,drug_name,gene_name,interaction_occurs_with_gene,interaction_type,evidence,gene_concept,gene_label,gene_match_type,drug_concept,drug_label,drug_match_type
20,16983347,imatinib,Bax,YES,ACTIVATING,Imatinib did not modulate the expression or ac...,normalize.gene.hgnc:959,BAX,100,normalize.therapy.rxcui:282388,imatinib,80


#### Identify Gene Pathways
Use KEGG and Entrez to identify the relevant biological pathways our target gene is involved in and also all the other neighbor genes present in that pathway in some form.

In [4]:
def get_gene_pathways(gene_symbol):
    """Return KEGG pathways (ID + common name) that a gene is part of."""
    mg = mygene.MyGeneInfo()
    query = mg.query(gene_symbol, fields="entrezgene", species="human")
    if not query['hits']:
        return []
    hit = query['hits'][0]
    entrez_id = hit.get('entrezgene')
    try:
        pathways = kegg.get_pathway_by_gene(entrez_id, "hsa")
        return [(pid, pname) for pid, pname in pathways.items()]
    except Exception:
        return []

def get_genes_in_pathway(pathway_id):
    """Fetch Entrez gene IDs for a KEGG pathway, handling cases with no GENE section."""
    pathway = kegg.get(pathway_id)
    parsed = kegg.parse(pathway)
    genes = []

    # Some KEGG pathways (like disease or resistance maps) have no GENE section.
    if 'GENE' in parsed:
        entries = parsed['GENE']
        if isinstance(entries, dict):
            # Modern KEGG format: {entrez_id: "symbol; description"}
            genes = list(entries.keys())
        elif isinstance(entries, list):
            # Legacy format: [entrez_id, "symbol; description", entrez_id, ...]
            genes = [entries[i] for i in range(0, len(entries), 2)]
    else:
        pass
    return genes

def build_entrez_to_symbol_mapper(entrez_ids):
    """
    Batch-resolve a set of Entrez IDs to symbols via MyGene,
    and return a function you can use in .apply().
    """
    # Unique, stringified, drop NAs
    unique_ids = sorted({str(i) for i in entrez_ids if pd.notna(i)})
    if not unique_ids:
        return lambda x: None

    res = mg.querymany(unique_ids, scopes="entrezgene", fields="symbol", species="human")

    # query -> symbol dict
    mapping = {r["query"]: r.get("symbol") for r in res if "query" in r}

    # The mapper handles either a scalar or a list/tuple
    def mapper(x):
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return None
        if isinstance(x, (list, tuple)):
            return [mapping.get(str(i)) for i in x]
        return mapping.get(str(x))
    return mapper

#### Identify Drug Targets
Utilize Chembl to identify known protein interactions for any target drug.

In [5]:
# TODO: OLD VERSION

def get_drug_target_symbols(drug_name):
    """
    Given a drug name (e.g., 'sunitinib' or 'ONC201'),
    return a DataFrame of all human SINGLE PROTEIN targets with gene symbols.
    """
    molecule = new_client.molecule
    target = new_client.target

    # 1. Find the ChEMBL ID of the drug
    mol_res = molecule.search(drug_name)
    if not mol_res:
        return pd.DataFrame()

    chembl_id = mol_res[0]["molecule_chembl_id"]

    # 2. Get all activities associated with this molecule
    activity = new_client.activity
    acts = activity.filter(molecule_chembl_id=chembl_id)

    target_ids = {a["target_chembl_id"] for a in acts if "target_chembl_id" in a}

    # 3. Pull target metadata and keep only relevant types
    records = []
    for tid in target_ids:
        rec = target.get(tid)
        if rec.get("organism") != "Homo sapiens":
            continue
        if rec.get("target_type") != "SINGLE PROTEIN":
            continue

        comps = rec.get("target_components", [])
        for comp in comps:
            acc = comp.get("accession")
            name = comp.get("component_name")
            gene_sym = comp.get("target_component_synonyms", [])
            gene_sym = [s["component_synonym"] for s in gene_sym if s["syn_type"] == "GENE_SYMBOL"]
            gene_sym = gene_sym[0] if gene_sym else None

            records.append({
                "target_chembl_id": tid,
                "uniprot_accession": acc,
                "protein_name": name,
                "gene_symbol": gene_sym,
                "pref_name": rec.get("pref_name")
            })

    df = pd.DataFrame(records)
    return df.drop_duplicates(subset=["gene_symbol"]).reset_index(drop=True)


def compute_drug_pathway_overlap(drug_targets_df, pathways_df):
    """
    For each pathway, check if any neighbor_symbol is in the set of known drug targets.
    Adds a boolean flag and the overlapping symbols.
    """
    target_genes = set(drug_targets_df["gene_symbol"].dropna().unique())
    
    def overlap_fn(row):
        n = row["neighbor_symbol"]
        if not n or pd.isna(n):
            return None
        if isinstance(n, list):
            overlap = list(set(n) & target_genes)
        else:
            overlap = [n] if n in target_genes else []
        return overlap if overlap else None
    

# TODO: New cache

from functools import lru_cache
import pandas as pd
import time

@lru_cache(maxsize=None)
def get_drug_target_symbols_cached(drug_name):
    """
    Cached + normalized wrapper around ChEMBL target retrieval.
    """
    if not isinstance(drug_name, str) or not drug_name.strip():
        return pd.DataFrame()

    normalized_name = drug_name.strip().lower()
    print(f"Fetching ChEMBL targets for '{normalized_name}'...")
    
    molecule = new_client.molecule
    target = new_client.target
    activity = new_client.activity

    # Try primary search
    mol_res = molecule.search(normalized_name)
    if not mol_res:
        print(f"⚠️ No ChEMBL hits for '{drug_name}'.")
        return pd.DataFrame()

    chembl_id = mol_res[0].get("molecule_chembl_id")
    if not chembl_id:
        print(f"⚠️ No ChEMBL ID for '{drug_name}'.")
        return pd.DataFrame()

    # Fetch activities
    acts = list(activity.filter(molecule_chembl_id=chembl_id))
    if not acts:
        print(f"⚠️ No activity data for {drug_name}.")
        return pd.DataFrame()

    # Collect unique target IDs
    target_ids = {a.get("target_chembl_id") for a in acts if "target_chembl_id" in a and a["target_chembl_id"]}
    if not target_ids:
        print(f"⚠️ No targets found for {drug_name}.")
        return pd.DataFrame()

    # Fetch metadata for each target
    records = []
    for tid in target_ids:
        try:
            rec = target.get(tid)
            if rec.get("organism") != "Homo sapiens":
                continue
            if rec.get("target_type") != "SINGLE PROTEIN":
                continue

            for comp in rec.get("target_components", []):
                acc = comp.get("accession")
                name = comp.get("component_name")
                gene_sym = None
                for s in comp.get("target_component_synonyms", []):
                    if s.get("syn_type") == "GENE_SYMBOL":
                        gene_sym = s.get("component_synonym")
                        break

                records.append({
                    "target_chembl_id": tid,
                    "uniprot_accession": acc,
                    "protein_name": name,
                    "gene_symbol": gene_sym,
                    "pref_name": rec.get("pref_name")
                })
        except Exception as e:
            print(f"Error retrieving target {tid} for {drug_name}: {e}")
            time.sleep(1)
            continue

    df = pd.DataFrame(records)
    if df.empty:
        print(f"⚠️ No protein targets for {drug_name}.")
    return df.drop_duplicates(subset=["gene_symbol"]).reset_index(drop=True)



#### Map Overlap
Identify genes in pathways that are known to be targeted by drug in predicted interaction. 

In [6]:
def compute_drug_pathway_overlap(drug_targets_df, pathways_df):
    """
    For each pathway, check if any neighbor_symbol is in the set of known drug targets.
    Adds a boolean flag and the overlapping symbols.
    """
    # --- Guard clause for invalid or missing input ---
    if (
        drug_targets_df is None
        or not isinstance(drug_targets_df, pd.DataFrame)
        or "gene_symbol" not in drug_targets_df.columns
        or drug_targets_df.empty
    ):
        # Return a copy with same expected output columns but all False/None
        result = pathways_df.copy()
        result["drug_target_overlap"] = None
        result["overlap_flag"] = False
        return result
    # ------------------------------------------------

    target_genes = set(drug_targets_df["gene_symbol"].dropna().unique())

    def overlap_fn(row):
        n = row["neighbor_symbol"]
        if not n or pd.isna(n):
            return None
        if isinstance(n, list):
            overlap = list(set(n) & target_genes)
        else:
            overlap = [n] if n in target_genes else []
        return overlap if overlap else None

    pathways_df = pathways_df.copy()
    pathways_df["drug_target_overlap"] = pathways_df.apply(overlap_fn, axis=1)
    pathways_df["overlap_flag"] = pathways_df["drug_target_overlap"].apply(lambda x: bool(x))
    return pathways_df


### Analysis
Put functions together to generate pathway targeting statistics

In [7]:
kegg = KEGG()
mg = MyGeneInfo()

df['#_additional_pathways_interactions'] = None
df['additional_pathway_interactions'] = None
df['#_additional_genes_in_pathways'] = None
df['additional_genes_in_pathways'] = None

# Identify all Signaling Pathways for Genes
pathways_df = pd.DataFrame()
for idx, row in tqdm(df.iterrows()):
    drug = row['drug_label']
    gene = row['gene_label']

    gene_pathways = get_gene_pathways(gene)

    for pathway in gene_pathways:
        row = {}
        neighbor_genes = get_genes_in_pathway(pathway[0])
        row['gene'] = gene
        row['pathway_id'] = pathway[0]
        row['pathway_label'] = pathway[1]
        row['neighbor_genes'] = neighbor_genes
        pathways_df = pd.concat([pathways_df, pd.DataFrame(row)]).reset_index(drop=True)

mapper = build_entrez_to_symbol_mapper(pathways_df['neighbor_genes'])
pathways_df['neighbor_symbol'] = pathways_df['neighbor_genes'].apply(mapper)

# Identify Drug Targets for Drugs
df['drug_targets'] = None
for idx, row in df.iterrows():
    drug = row['drug_label']
    try:
        targets_df = get_drug_target_symbols_cached(drug)
        df.at[idx, 'drug_targets'] = targets_df if not targets_df.empty else None
    except Exception as e:
        print(f"❌ Failed for {drug}: {e}")

# Map Drug Targets to Pathways, Quantify
for idx, row in df.iterrows():
    drug_targets = row['drug_targets']
    pathways_with_overlap = compute_drug_pathway_overlap(drug_targets,pathways_df)
    answers = pathways_with_overlap[(pathways_with_overlap['overlap_flag']) & (~pathways_with_overlap['neighbor_symbol'].str.contains(row['gene_name']))]

    num_genes = len(set(answers['neighbor_symbol']))
    num_pathways = len(set(answers['pathway_label']))
    pathways = set(answers['pathway_label'])
    genes = set(answers['neighbor_symbol'])

    df.loc[idx, '#_additional_pathways_interactions'] = num_pathways
    df.at[idx, 'additional_pathway_interactions'] = list(pathways)
    df.loc[idx, '#_additional_genes_in_pathways'] = num_genes
    df.at[idx, 'additional_genes_in_pathways'] = list(genes)


0it [00:00, ?it/s]Input sequence provided is already in string format. No operation performed
1it [00:06,  6.14s/it]Input sequence provided is already in string format. No operation performed
2it [00:10,  5.34s/it]Input sequence provided is already in string format. No operation performed
3it [00:21,  7.78s/it]Input sequence provided is already in string format. No operation performed
4it [01:32, 32.71s/it]Input sequence provided is already in string format. No operation performed
5it [02:41, 45.75s/it]Input sequence provided is already in string format. No operation performed
6it [03:50, 53.79s/it]Input sequence provided is already in string format. No operation performed
7it [04:04, 40.84s/it]Input sequence provided is already in string format. No operation performed
8it [04:19, 32.52s/it]Input sequence provided is already in string format. No operation performed
9it [04:34, 26.86s/it]Input sequence provided is already in string format. No operation performed
10it [04:35, 19.08s/it]I

Fetching ChEMBL targets for 'venetoclax'...
Fetching ChEMBL targets for 'glutathione'...
Fetching ChEMBL targets for 'kynurenine'...
Fetching ChEMBL targets for 'onc201'...
Fetching ChEMBL targets for 'trehalose'...
Fetching ChEMBL targets for 'metformin'...
Fetching ChEMBL targets for 'ampelopsin'...
⚠️ No protein targets for AMPELOPSIN.
Fetching ChEMBL targets for 'salubrinal'...
Fetching ChEMBL targets for 'cisplatin'...
⚠️ No protein targets for cisplatin.
Fetching ChEMBL targets for 'therapeutic androgen'...
⚠️ No activity data for Therapeutic Androgen.
Fetching ChEMBL targets for 'enzalutamide'...
Fetching ChEMBL targets for 'ganetespib'...
Fetching ChEMBL targets for 'andrographolide'...
Fetching ChEMBL targets for 'phenformin'...
Fetching ChEMBL targets for 'imatinib'...
Fetching ChEMBL targets for 'tamoxifen'...
Fetching ChEMBL targets for 'docetaxel anhydrous'...
Fetching ChEMBL targets for 'quizartinib'...
Fetching ChEMBL targets for 'gilteritinib'...
Fetching ChEMBL targets

In [8]:
# Save
df.to_excel('results.xlsx')

### Inspect

In [9]:
inspect = df[['gene_label','drug_label','#_additional_genes_in_pathways','additional_genes_in_pathways','#_additional_pathways_interactions']].sort_values(by='#_additional_pathways_interactions',ascending=False).reset_index(drop=True)
inspect[0:10]

,gene_label,drug_label,#_additional_genes_in_pathways,additional_genes_in_pathways,#_additional_pathways_interactions
0,BAX,imatinib,421,"[MYH10, MAP3K13, DHCR24, PI4KB, DAPK2, ABCC4, ...",204
1,BCL2L11,gefitinib,385,"[MAP3K13, DHCR24, PI4KB, DAPK2, ABCC4, PIK3C2B...",201
2,PANDAR,paclitaxel,166,"[BDKRB2, EDNRB, OPRM1, ABCC4, PTPRC, MAOA, HTR...",200
3,EGFR,gefitinib,384,"[MAP3K13, DHCR24, PI4KB, DAPK2, ABCC4, PIK3C2B...",200
4,TP53,apigenin,158,"[OPRM1, MAOA, ALOX15, CXCL8, ALOX5, DAPK3, PDK...",200
5,KPNA4,paclitaxel,166,"[BDKRB2, EDNRB, OPRM1, ABCC4, PTPRC, MAOA, HTR...",200
6,BTK,ibrutinib,267,"[MYH10, DHCR24, GNAI2, PRKG1, OPRM1, MCM4, MAO...",198
7,BTK,ibrutinib,267,"[MYH10, DHCR24, GNAI2, PRKG1, OPRM1, MCM4, MAO...",198
8,EGFR,vemurafenib,270,"[MYH10, BDKRB2, DHCR24, GNAI2, PRKG1, OPRM1, M...",195
9,BCL2A1,gilteritinib,225,"[DHCR24, GNAI2, PRKG1, OPRM1, MCM4, MAOA, HTR1...",195


In [10]:
value = 3
print(f'AI Curated Interaction: {inspect['drug_label'][value]} / {inspect['gene_label'][value]}')
print(f'# of genes that neighbor {inspect['gene_label'][value]} in known signaling pathways: {inspect['#_additional_genes_in_pathways'][value]}')
print(f'# of neighbor genes that have known interactions with {inspect['drug_label'][value]}: {inspect['#_additional_pathways_interactions'][value]}')

AI Curated Interaction: gefitinib / EGFR
# of genes that neighbor EGFR in known signaling pathways: 384
# of neighbor genes that have known interactions with gefitinib: 200


TODO: Why do some rows have more neighbor interactions than genes in pathways?
TODO: Graphs